<a href="https://colab.research.google.com/github/zhukuixi/d2l/blob/main/d2l_LinearRegresion_fromScratchfromAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
import random

# From Scratch

In [ ]:
def synthetic_data(w,b,n):
  # X^2*w1 + X*w2 +b
  X = torch.normal(0,1,(n,len(w)))  
  X[:,0] = X[:,0]**2
  Y = X@w+b
  Y += torch.normal(0,0.01,Y.shape)
  return X,Y

In [154]:
# generate synthetic data
true_para_w = torch.tensor([3,2],dtype=torch.float32).reshape(-1,1)
true_para_b = torch.tensor([1])
X,Y = synthetic_data(true_para_w,true_para_b,100)

# batch generator
def batchIterator(x,y,batch_size):
  index = np.arange(len(x))
  random.shuffle(index)
  
  for i in range(0,len(x),batch_size):
    batch_index = index[np.arange(i,min(i+batch_size,len(x)))]
    yield x[batch_index],y[batch_index]
  


# model
def net(X,w,b):
  return X@w+b

# loss
def getloss(para,X,Y):
  w,b = para
  y_predict = net(X,w,b)
  loss = ((y_predict-Y.reshape(y_predict.shape))**2).mean().sqrt()
  return loss

# update parameter
def updatePara(para,lr,batch_size):
  for p in para:
      p.data -= p.grad*lr/batch_size
      p.grad.zero_()
  


In [155]:
# initial parameter
w = torch.normal(0,1,true_para_w.shape,requires_grad=True)
b = torch.normal(0,1,true_para_b.shape,requires_grad=True)
batch_size=20
lr=0.5
for epoch in range(30):
  for x,y in batchIterator(X,Y,batch_size):    
    loss = getloss((w,b),x,y)
    loss.backward()
    updatePara((w,b),lr,batch_size)
  with torch.no_grad():
    trainLoss = getloss((w,b),X,Y)
    print(f'epoch:{epoch} train_loss{trainLoss}')
 

epoch:0 train_loss6.6645097732543945
epoch:1 train_loss6.3247199058532715
epoch:2 train_loss5.984911918640137
epoch:3 train_loss5.654780387878418
epoch:4 train_loss5.323352336883545
epoch:5 train_loss4.99605655670166
epoch:6 train_loss4.6825337409973145
epoch:7 train_loss4.3686137199401855
epoch:8 train_loss4.063063621520996
epoch:9 train_loss3.7643165588378906
epoch:10 train_loss3.4803380966186523
epoch:11 train_loss3.2016892433166504
epoch:12 train_loss2.9322237968444824
epoch:13 train_loss2.6777453422546387
epoch:14 train_loss2.4334075450897217
epoch:15 train_loss2.207381010055542
epoch:16 train_loss1.9941487312316895
epoch:17 train_loss1.801249623298645
epoch:18 train_loss1.6251888275146484
epoch:19 train_loss1.4626107215881348
epoch:20 train_loss1.3148353099822998
epoch:21 train_loss1.1777992248535156
epoch:22 train_loss1.0511094331741333
epoch:23 train_loss0.9338873028755188
epoch:24 train_loss0.8254746198654175
epoch:25 train_loss0.7234444618225098
epoch:26 train_loss0.627833962

# From API

In [ ]:
from torch.utils import data

In [ ]:
# batch generator
def load_array(data_array,batch_size,is_train=True):
  dataset = data.TensorDataset(*data_array)
  return data.DataLoader(dataset,batch_size=batch_size,shuffle=is_train)

In [ ]:
batch_size=20
data_iter = load_array((X, Y), batch_size)

In [ ]:
from torch import nn

# batch generator
def load_array(data_array,batch_size,is_train=True):
  dataset = data.TensorDataset(*data_array)
  return data.DataLoader(dataset,batch_size=batch_size,shuffle=is_train)

#model
net = nn.Sequential(nn.Linear(2, 1))

#loss
loss = nn.HuberLoss()

#optimizer
trainer = torch.optim.SGD(net.parameters(), lr=0.03)


In [ ]:

#initial parameter
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

num_epochs = 20
for epoch in range(num_epochs):
    for x, y in data_iter:
        l = loss(net(x) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(X), Y)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 3.887452
epoch 2, loss 3.665075
epoch 3, loss 3.446028
epoch 4, loss 3.230402
epoch 5, loss 3.017489
epoch 6, loss 2.807659
epoch 7, loss 2.600961
epoch 8, loss 2.398503
epoch 9, loss 2.199972
epoch 10, loss 2.006513
epoch 11, loss 1.819197
epoch 12, loss 1.639069
epoch 13, loss 1.467265
epoch 14, loss 1.302925
epoch 15, loss 1.147366
epoch 16, loss 1.000661
epoch 17, loss 0.862736
epoch 18, loss 0.734626
epoch 19, loss 0.618156
epoch 20, loss 0.513352
